In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [2]:
import urllib.parse
from bs4 import BeautifulSoup

In [3]:
import requests
import brotli

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
filein_path = "../../../data/data_wikis/1 - politician data for test scraping/"

In [6]:
fileout_path = "../../../data/data_wikis/2.1 - sk namuwiki scraping test/"

In [7]:
logfile_name = "_seleniumtest_log.txt"

# import sk & tw politician test data

In [8]:
sktest = pd.read_excel(filein_path+"sk_testset.xlsx")

In [9]:
sktest.shape

(9960, 24)

In [10]:
# sktest["URL_NamuWiki"]=np.nan

In [11]:
sktest[100:200]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
100,강선구,姜善求,19490218,남,65,구·시·군의회의원선거,20140604,Legislator,대전광역시,서구,...,등록,0,NaN,NaN,NaN,무직,미기재,(전)둔산상포장의사대표,(전)월평지역의용소방대장,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EA%B5%AC
101,강선우,姜仙祐,19780602,여,37,비례대표국회의원선거,20160413,Party List Legislator,전국,전국,...,등록,0,NaN,NaN,NaN,교수,미국 위스콘신대 가족학 박사(6년),(현)사우스다코타주립대 교수,(현)전미 노인학회 정회원,NaN
102,강선화,姜善花,19800818,여,33,시·도의회의원선거,20140604,Legislator,서울특별시,영등포구,...,등록,0,NaN,NaN,NaN,정당인,전북대학교 심리학과 졸업,(전)민주노총 금속노조 선전부장,(현)영등포 뉴타운폐지와 마을공동체 추진포럼 운영위원,False
103,강성균,姜性均,19520223,남,62,교육의원선거,20140604,Education Council Member,제주특별자치도,제주시,...,등록,1,44340.0,57.37,NaN,교육활동,제주대학교 교육대학원(석사) 교육학과 국어교육전공 취득,(전) 제주과학고등학교 교장,(전) 탐라교육원장,False
104,강성길,姜成吉,19630823,남,50,구·시·군의회의원선거,20140604,Legislator,서울특별시,서초구,...,등록,0,NaN,NaN,NaN,제6대 서초구의원,중앙대학교 사회개발 대학원 사회복지학과 석사과정 재학,(현)제6대 서초구의회 행정복지위원장,(현)세금바로쓰기 납세자 운동 서울시지부 회장,False
105,강성길,姜誠吉,19570808,남,56,구·시·군의회의원선거,20140604,Legislator,대구광역시,동구,...,등록,0,NaN,NaN,NaN,직장인,중앙대학교 사회개발대학원 보건행정학과 석사 수료,(전)새정치민주연합 중앙당 정책당원,(현)민주노총 및 공공운수노조 대의원,False
106,강성민,姜成旼,19710806,남,42,시·도의회의원선거,20140604,Legislator,제주특별자치도,제주시,...,등록,0,NaN,NaN,NaN,정치인,제주대학교 법정대학 행정학과 졸업,(현)제주미래비전연구원 제주생활정책포럼 대표,(전)제주특별자치도의회 정책자문위원,https://namu.wiki/w/%EA%B0%95%EC%84%B1%EB%AF%BC
107,강성봉,姜聖峰,19710201,남,43,광역의원비례대표선거,20140604,Party List Legislator,전라북도,NaN,...,등록,0,NaN,NaN,NaN,회사원,고려대학교 행정대학원 졸업,(현)새정치민주연합전국청년위원회부위원장,NaN,False
108,강성수,姜聲守,19550629,남,58,구·시·군의회의원선거,20140604,Legislator,인천광역시,남동구,...,등록,0,NaN,NaN,NaN,자영업,서정대학교 사회복지과 재학,(전)간석풍림아파트 입주자대표회의 회장,(현)인천시 호남향우회 본회 자문위원,https://namu.wiki/w/%EA%B0%95%EC%84%B1%EC%88%98
109,강성언,姜成彦,19530207,남,61,시·도의회의원선거,20140604,Legislator,서울특별시,강북구,...,등록,1,18274.0,52.64,NaN,건축사,건국대학교 공과대학 건축공학과 졸업,(전)서울시 교육청 사무관,(전)민주당 강북을 부위원장,False


In [12]:
# percent duplicates

1 - (sktest["Name_Native"].drop_duplicates().shape[0] / sktest.shape[0])

0.12550200803212852

# scrape namuwiki with selenium

In [13]:
url_stem = "https://namu.wiki"

p_error_name = "data-v-1ac09f2e"
p_error_text = "해당 문서를 찾을 수 없습니다."

In [14]:
# instantiate selenium webdriver
service = Service(executable_path=ChromeDriverManager().install())

# scrape namuwiki using selenium
for index,p in sktest.iterrows():
    
    if(pd.isnull(p.URL_NamuWiki)):
    
        # construct url
        url= url_stem+"/w/"+urllib.parse.quote(p.Name_Native.encode("utf-8"))

        # reset content & has_content_flag
        has_content_flag = True
        content=""

        try:

            # request url using selenium webdriver
            driver = webdriver.Chrome(service=service)
            driver.get(url)

            # check if page has content
            elements = driver.find_elements(By.TAG_NAME, 'p')

            for e in elements:
                if(e.text==p_error_text):
                    has_content_flag = False

            if(has_content_flag):
                content = driver.page_source

            # close webdriver
            driver.quit()

            # save content
            if(has_content_flag):

                # save output as an html file
                fout = open(fileout_path+"seleniumtest_"+str(index)+"_"+p.Name_Native+".html","w",encoding="utf-8")
                fout.write(content)
                fout.close()

                # update HasNamuWiki content field in dataframe
                #sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=url

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,url,"\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Has Content")

            else:

                # update HasNamuWiki content field in dataframe
                # sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=False

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,"False","\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Empty")
            
        except KeyboardInterrupt:
        
            print("Keyboard Interrupt")
            
            break

        except Exception as e:

            print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Error")
            print(e)





101 강선우 https://namu.wiki/w/%EA%B0%95%EC%84%A0%EC%9A%B0 Number of <p> elements: 4 Has Content
513 권선택 https://namu.wiki/w/%EA%B6%8C%EC%84%A0%ED%83%9D Number of <p> elements: 4 Has Content
1874 김은지 https://namu.wiki/w/%EA%B9%80%EC%9D%80%EC%A7%80 Number of <p> elements: 4 Has Content
1926 김인순 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EC%88%9C Number of <p> elements: 4 Has Content
1980 김재기 https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EA%B8%B0 Number of <p> elements: 4 Has Content
1987 김재선 https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EC%84%A0 Number of <p> elements: 4 Has Content
2080 김정진 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%A7%84 Number of <p> elements: 4 Has Content
2095 김정희 https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC Number of <p> elements: 4 Has Content
2096 김정희 https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC Number of <p> elements: 4 Has Content
2097 김정희 https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC Number of <p> elements: 4 Has Content
2098 김정희 https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%

2175 김종익 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%9D%B5 Number of <p> elements: 4 Has Content
2176 김종익 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%9D%B5 Number of <p> elements: 4 Has Content
2177 김종인 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%9D%B8 Number of <p> elements: 4 Has Content
2178 김종주 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%A3%BC Number of <p> elements: 5 Empty
2179 김종진 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%A7%84 Number of <p> elements: 4 Has Content
2180 김종찬 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%B0%AC Number of <p> elements: 4 Has Content
2181 김종천 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%B2%9C Number of <p> elements: 4 Has Content
2182 김종천 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%B2%9C Number of <p> elements: 4 Has Content
2183 김종천 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%B2%9C Number of <p> elements: 4 Has Content
2184 김종철 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%B2%A0 Number of <p> elements: 4 Has Content
2185 김종철 https://namu.wiki/w/%EA%B9%80%EC%A2%85%EC%B2%A0

2263 김중훈 https://namu.wiki/w/%EA%B9%80%EC%A4%91%ED%9B%88 Number of <p> elements: 5 Empty
2264 김중희 https://namu.wiki/w/%EA%B9%80%EC%A4%91%ED%9D%AC Number of <p> elements: 4 Has Content
2265 김중희 https://namu.wiki/w/%EA%B9%80%EC%A4%91%ED%9D%AC Number of <p> elements: 4 Has Content
2266 김지근 https://namu.wiki/w/%EA%B9%80%EC%A7%80%EA%B7%BC Number of <p> elements: 5 Empty
2267 김지덕 https://namu.wiki/w/%EA%B9%80%EC%A7%80%EB%8D%95 Number of <p> elements: 5 Empty
2268 김지동 https://namu.wiki/w/%EA%B9%80%EC%A7%80%EB%8F%99 Number of <p> elements: 5 Empty
2269 김지수 https://namu.wiki/w/%EA%B9%80%EC%A7%80%EC%88%98 Number of <p> elements: 4 Has Content
2270 김지수 https://namu.wiki/w/%EA%B9%80%EC%A7%80%EC%88%98 Number of <p> elements: 4 Has Content
2271 김지수 https://namu.wiki/w/%EA%B9%80%EC%A7%80%EC%88%98 Number of <p> elements: 4 Has Content
2272 김지식 https://namu.wiki/w/%EA%B9%80%EC%A7%80%EC%8B%9D Number of <p> elements: 5 Empty
2273 김지연 https://namu.wiki/w/%EA%B9%80%EC%A7%80%EC%97%B0 Number of <p> elements:

2351 김찬술 https://namu.wiki/w/%EA%B9%80%EC%B0%AC%EC%88%A0 Number of <p> elements: 5 Empty
2352 김찬심 https://namu.wiki/w/%EA%B9%80%EC%B0%AC%EC%8B%AC Number of <p> elements: 5 Empty
2353 김찬우 https://namu.wiki/w/%EA%B9%80%EC%B0%AC%EC%9A%B0 Number of <p> elements: 4 Has Content
2354 김찬주 https://namu.wiki/w/%EA%B9%80%EC%B0%AC%EC%A3%BC Number of <p> elements: 5 Empty
2355 김찬진 https://namu.wiki/w/%EA%B9%80%EC%B0%AC%EC%A7%84 Number of <p> elements: 4 Has Content
2356 김창관 https://namu.wiki/w/%EA%B9%80%EC%B0%BD%EA%B4%80 Number of <p> elements: 5 Empty
2357 김창구 https://namu.wiki/w/%EA%B9%80%EC%B0%BD%EA%B5%AC Number of <p> elements: 5 Empty
2358 김창규 https://namu.wiki/w/%EA%B9%80%EC%B0%BD%EA%B7%9C Number of <p> elements: 4 Has Content
2359 김창규 https://namu.wiki/w/%EA%B9%80%EC%B0%BD%EA%B7%9C Number of <p> elements: 4 Has Content
2360 김창규 https://namu.wiki/w/%EA%B9%80%EC%B0%BD%EA%B7%9C Number of <p> elements: 4 Has Content
2361 김창규 https://namu.wiki/w/%EA%B9%80%EC%B0%BD%EA%B7%9C Number of <p> elements:

2440 김충식 https://namu.wiki/w/%EA%B9%80%EC%B6%A9%EC%8B%9D Number of <p> elements: 4 Has Content
2441 김충영 https://namu.wiki/w/%EA%B9%80%EC%B6%A9%EC%98%81 Number of <p> elements: 5 Empty
2442 김충호 https://namu.wiki/w/%EA%B9%80%EC%B6%A9%ED%98%B8 Number of <p> elements: 5 Empty
2443 김치백 https://namu.wiki/w/%EA%B9%80%EC%B9%98%EB%B0%B1 Number of <p> elements: 5 Empty
2444 김치환 https://namu.wiki/w/%EA%B9%80%EC%B9%98%ED%99%98 Number of <p> elements: 5 Empty
2445 김탁 https://namu.wiki/w/%EA%B9%80%ED%83%81 Number of <p> elements: 5 Empty
2446 김태경 https://namu.wiki/w/%EA%B9%80%ED%83%9C%EA%B2%BD Number of <p> elements: 4 Has Content
2447 김태경 https://namu.wiki/w/%EA%B9%80%ED%83%9C%EA%B2%BD Number of <p> elements: 4 Has Content
2448 김태국 https://namu.wiki/w/%EA%B9%80%ED%83%9C%EA%B5%AD Number of <p> elements: 5 Empty
2449 김태규 https://namu.wiki/w/%EA%B9%80%ED%83%9C%EA%B7%9C Number of <p> elements: 4 Has Content
2450 김태균 https://namu.wiki/w/%EA%B9%80%ED%83%9C%EA%B7%A0 Number of <p> elements: 4 Has Content
2

In [ ]:
# things to check

# 0. drop duplicates. some duplicate pages are different (at least size differs). why?  

# 0.1 drop duplicates (by=[Name_Native, Name_Chinese, Birthdate]) in TW dataset, before scraping

# 1. recorded links are valid (no false positives); non-recorded links are actually empty (no false negatives)

# 2. disambiguation? what about names that link to more than one person?
#
#       - disambiguation links are usually included in the page
#
#       - example
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD - 강민국
#               정치인에 대한 내용은 강민국(정치인) 문서를 참고하십시오.
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD(%EC%A0%95%EC%B9%98%EC%9D%B8) - 강민국(정치인)
#
#       - another example
#               https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A7%84

# update sktest with selenium log from latest run

In [15]:
log_columns = ["index","Name_Native","url","dropme"]
log = pd.read_csv(fileout_path+logfile_name,header=None,encoding="utf-8",names=log_columns)
log.drop(columns=["dropme"],inplace=True)
log

,index,Name_Native,url
0,101,강선우,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EC%9A%B0
1,513,권선택,https://namu.wiki/w/%EA%B6%8C%EC%84%A0%ED%83%9D
2,1874,김은지,https://namu.wiki/w/%EA%B9%80%EC%9D%80%EC%A7%80
3,1926,김인순,https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EC%88%9C
4,1980,김재기,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EA%B8%B0
5,1987,김재선,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EC%84%A0
6,2080,김정진,https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%A7%84
7,2095,김정희,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC
8,2096,김정희,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC
9,2097,김정희,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC


In [16]:
sktest.shape

(9960, 24)

In [17]:
sktest_updated = sktest.merge(log,how="left",on="Name_Native")
sktest_updated.shape

(10424, 26)

In [18]:
# update URL_NamuWiki with log results
sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"URL_NamuWiki"] = sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"url"]

# confirm that changes have been made
sktest_updated.loc[sktest_updated.url.notnull(),["URL_NamuWiki","url"]]

,URL_NamuWiki,url
101,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EC%9A%B0,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EC%9A%B0
513,https://namu.wiki/w/%EA%B6%8C%EC%84%A0%ED%83%9D,https://namu.wiki/w/%EA%B6%8C%EC%84%A0%ED%83%9D
1874,https://namu.wiki/w/%EA%B9%80%EC%9D%80%EC%A7%80,https://namu.wiki/w/%EA%B9%80%EC%9D%80%EC%A7%80
1926,https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EC%88%9C,https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EC%88%9C
1980,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EA%B8%B0,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EA%B8%B0
1987,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EC%84%A0,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EC%84%A0
2080,https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%A7%84,https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%A7%84
2095,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC
2096,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC
2097,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC


In [19]:
# change "False" strings to boolean False
sktest_updated.loc[sktest_updated.URL_NamuWiki=="False","URL_NamuWiki"]=False

In [20]:
sktest_updated[sktest_updated["index"].notnull()]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki,index,url
101,강선우,姜仙祐,19780602,여,37,비례대표국회의원선거,20160413,Party List Legislator,전국,전국,...,NaN,NaN,NaN,교수,미국 위스콘신대 가족학 박사(6년),(현)사우스다코타주립대 교수,(현)전미 노인학회 정회원,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EC%9A%B0,101.0,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EC%9A%B0
513,권선택,權善宅,19551201,남,58,시·도지사선거,20140604,Chief,대전광역시,NaN,...,322762.0,50.07,NaN,정당인,대전대학교대학원 행정학과 졸업(행정학 박사),"(전) 17, 18대 국회의원",(전) 청와대 인사비서관,https://namu.wiki/w/%EA%B6%8C%EC%84%A0%ED%83%9D,513.0,https://namu.wiki/w/%EA%B6%8C%EC%84%A0%ED%83%9D
1874,김은지,金恩志,19880307,여,26,광역의원비례대표선거,20140604,Party List Legislator,강원도,NaN,...,NaN,NaN,NaN,시민단체 활동가,강원대학교 산림환경과학대학 산림경영·조경학부 산림경영학전공,(전)(사)춘천생명의 숲,NaN,https://namu.wiki/w/%EA%B9%80%EC%9D%80%EC%A7%80,1874.0,https://namu.wiki/w/%EA%B9%80%EC%9D%80%EC%A7%80
1926,김인순,金仁順,19570103,여,57,기초의원비례대표선거,20140604,Party List Legislator,강원도,원주시,...,NaN,NaN,NaN,정당인,"한성디지털대학교 미용예술과 졸업(현,서울문화예술대학교)",(전)상지여고 총동문회장,(전)새누리당 원주갑 여성지회장,https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EC%88%9C,1926.0,https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EC%88%9C
1980,김재기,金在起,19510401,남,63,구·시·군의회의원선거,20140604,Legislator,인천광역시,중구,...,NaN,NaN,NaN,중구의회의원,평생교육법에 의거 학력이 지정된 학력인정 남인천고등학교(2년제) 졸업,(현)민주평화통일자문회의 중구협의회 간사,(전)제5대 인천광역시 중구의회의원,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EA%B8%B0,1980.0,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EA%B8%B0
1987,김재선,金在善,19600325,남,54,구·시·군의회의원선거,20140604,Legislator,부산광역시,수영구,...,NaN,NaN,NaN,서울광고나염 대표,정산중학교 졸업,(현)남천1동 당무회장,(전)수영구 바르게살기 사무국장,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EC%84%A0,1987.0,https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EC%84%A0
2080,김정진,金禎鎭,19710905,남,42,시·도의회의원선거,20140604,Legislator,대전광역시,유성구,...,NaN,NaN,NaN,정당인,충남대학교 건축공학과 졸업,(전)진보신당 충남도당 사무처장,(현)노동당 대전시당 민영화저지위원장,https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%A7%84,2080.0,https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%A7%84
2095,김정희,金楨姬,19770412,여,37,시·도의회의원선거,20140604,Legislator,경기도,수원시권선구,...,NaN,NaN,NaN,정당인,목포여자상업고등학교 졸업,(현) 학교비정규직노동조합 수원지회 사무국장,(현) 민주노총 경기본부 대의원,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC,2095.0,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC
2096,김정희,金楨姬,19770412,여,37,시·도의회의원선거,20140604,Legislator,경기도,수원시권선구,...,NaN,NaN,NaN,정당인,목포여자상업고등학교 졸업,(현) 학교비정규직노동조합 수원지회 사무국장,(현) 민주노총 경기본부 대의원,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC,2096.0,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC
2097,김정희,金楨姬,19770412,여,37,시·도의회의원선거,20140604,Legislator,경기도,수원시권선구,...,NaN,NaN,NaN,정당인,목포여자상업고등학교 졸업,(현) 학교비정규직노동조합 수원지회 사무국장,(현) 민주노총 경기본부 대의원,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC,2097.0,https://namu.wiki/w/%EA%B9%80%EC%A0%95%ED%9D%AC


In [21]:
# drop log column (url)
sktest_updated.drop(columns=["index","url"],inplace=True)

In [22]:
# drop duplicate rows
sktest_updated.drop_duplicates().shape

(9960, 24)

In [23]:
# drop duplicate politicians
sktest_updated.sort_values(["Name_Native","Name_Chinese","Birthdate","ElectionDate"],inplace=True)
sktest_updated.drop_duplicates(["Name_Native","Name_Chinese","Birthdate"],keep="last",inplace=True)

In [24]:
sktest_updated.shape

(9960, 24)

In [25]:
# refresh and save updated sktest
sktest = sktest_updated
sktest.to_excel(filein_path+"sk_testset.xlsx",index=False)

In [26]:
# delete log file
os.remove(fileout_path+logfile_name)

In [33]:
# DEPRECATED - assign URL_NamuWiki link based on files already scraped - I needed this before I created a log file

# import os
# filelist = os.listdir(fileout_path)
# test = sktest
# url_stem = "https://namu.wiki"
# test[100:200]
# scraped = [f.split("_")[2].split(".")[0].strip() for f in filelist]
# for s in scraped:
#     url = url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8"))
#     # print(url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8")))
#     test.loc[test.Name_Native==s,"URL_NamuWiki"]=url
# test[100:200]